# 1. Define your Diff function in python.

In [8]:
import math
import random
import matplotlib.pyplot as plt
import numpy as np
random.seed(42)


# pick three vectors corresponding to letter of each group members name
u = (59, 94)
v = (62, 60)
w = (10, 40)


# define distance function 
def distance(vector_1, vector_2):
    ''' this function takes in two vectors and calculates and returns the distance between them'''
    
    return math.log(3 + 3 * (vector_1[0] - vector_2[0]) ** 2 + 1.5 * (vector_1[1] - vector_2[1]) ** 2)


# define the overall difference function
def difference(input_vector):
    ''' this function takes in a vector, compares it to the three fixed vectors for our group
    and returns the overall difference '''
    
    return distance(input_vector, u) + distance(input_vector, v) + distance(input_vector, w)


# 2. Compute Diff(x) for 20 random points. 
**What is the average value of Diff?**<br>
The average value was 24.91

**What is the lowest and highest value you found?**<br>
Lowest value was 21.63 and the highest was 27.21

**Is random search a good way to minimize this function?**<br>
No. Random search literally just gives you random points that you can then test the value of. There is no learning from the results of previously tried points and the time to get to a relatively good solution could take a very long time. There is no order and you are not "approaching" the minimal value, you are hoping that you happen to get a value that is close to the minimum by pure chance.

In [9]:
# randomly generate 20 points within the range
random_points = [(random.uniform(0, 100), random.uniform(0, 100)) for _ in range(20)]

# obtain the difference values for each of the 20 randomly generated points
diff_values = [difference(point) for point in random_points]

# obtain and print the average, highest and lowest value
average_diff = np.mean(diff_values)
max_diff = np.max(diff_values)
min_diff = np.min(diff_values)
print(f'Average difference: {average_diff}\nMaximum value: {max_diff}\nMinimum value: {min_diff}')


Average difference: 25.17733489812628
Maximum value: 28.168531397459258
Minimum value: 21.27284641900745


# 3. Set xzero=(50,50). Compute Diff(x)=Diff(50,50).

In [10]:
# create xzero and test with the difference function
xzero = (50, 50)
xzero_diff = difference(xzero)
print(xzero_diff)


22.93451831163977


# 4. Create a function to compute and print the gradient of Diff by using a small delta=0.001.

In [11]:
# create a function that calculates the gradient of the difference
def gradient_diff(input_vector, delta = 0.001):
    ''' this function takes in an input_vector and calculates its gradient in the 
    difference function '''
    
    # alter x_1 by delta then calculate the first partial derivative (gradient of x1)
    altered_x1_vector = (input_vector[0] + delta, input_vector[1])
    x1_gradient = (difference(altered_x1_vector) - difference(input_vector)) / delta
    
    # alter x_2 by delta then calculate the second partial derivative (gradient of x2)
    altered_x2_vector = (input_vector[0], input_vector[1]  + delta)
    x2_gradient = (difference(altered_x2_vector) - difference(input_vector)) / delta

    return (x1_gradient, x2_gradient)
    


# 5. Use this function to print the gradient at (0,0), (100,0), (0,100) and (100,100). 
**Based on these values, is it likely that the minimum of the function is inside this area? And the maximum?**
Based on the gradients, it is likely that the minimum is inside the area, since the gradients are pointing inwards from the corners.
Based on the gradients it is likely that there is no maximum in this region. At point (100,100) the gradient is positive in both directions, suggesting an increase in the function moving towards that point. Still, the gradients do not consistently point outwards suggesting that there is no maximum inside the specified region.

In [23]:
vectors = [(0,0), (100,0), (0,100), (100,100)]
delta = 0.001
for i in vectors:
    print(f"Gradient at {i}: {gradient_diff(i, delta)}")

Gradient at (0, 0): (-0.05909979456575343, -0.06692313176159814)
Gradient at (100, 0): (0.05708573236873349, -0.03839370529234998)
Gradient at (0, 100): (-0.070930388567092, 0.0418880083330464)
Gradient at (100, 100): (0.10026751570535453, 0.02740716165305912)


# 6. Define xzero=(50,50) and stepsize=1.0. 
**Make a function to compute xnext by taking e a small step into the opposite direction of the gradient, so that diff decreases. E.g. if the gradient is (0.4,0.6), do xnext=(oldx1-0.4*stepsize,oldx2-0.6*stepsize**

In [52]:
xzero = (50,50)
stepsize = 0.1
delta = 0.001
def xnext(input_vector, delta, stepsize):
    """
    xnext Function:
    Adjusts the input vector by taking steps in the opposite direction of the gradient, aiming to minimize the function value.
    """
    gradient = gradient_diff(input_vector, delta)
    x1next = input_vector[0] - gradient[0] * stepsize
    x2next = input_vector[1] - gradient[1] * stepsize
    return (x1next, x2next)
xnext(xzero, delta, stepsize)

(50.009176650724996, 50.008712874604555)

# 7. Repeat this multiple times (make a function to do this automatically) and print x and Diff(x) at each step. 
**You may need to adjust stepsize if you overshoot or are not moving at all. Try to reach a local minimum.**

In [78]:
def g_descent(input_vector, stepsize, delta, iters):
    """
    Gradient Descent Function:
    
    This function performs gradient descent optimization to find the local minimum of a given function `Diff(x)`.
    It iteratively adjusts the input vector by taking steps in the opposite direction of the gradient, aiming to minimize `Diff(x)`.
    """
    current = input_vector
    for i in range(iters):
        x_next = xnext(current, delta, stepsize)
        dif = difference(x_next)
        print(f"Step {i+1}: x = {x_next}, Diff(x) = {dif}")
        current = x_next

#testing
xzero = (50,50)
stepsize = 0.6
delta = 0.001
iters = 200
g_descent(xzero, stepsize, delta, iters)
#at step 136 local minimum was reached.


Step 1: x = (50.055059904349974, 50.05227724762733), Diff(x) = 22.924886120740425
Step 2: x = (50.110496999364784, 50.10472604355556), Diff(x) = 22.915154038110227
Step 3: x = (50.16631618492866, 50.15734908402152), Diff(x) = 22.905320092616716
Step 4: x = (50.22252248218564, 50.21014912940913), Diff(x) = 22.895382255128283
Step 5: x = (50.27912103781347, 50.26312900654517), Diff(x) = 22.885338436172894
Step 6: x = (50.336117128472324, 50.31629161110587), Diff(x) = 22.87518648347882
Step 7: x = (50.393516165500785, 50.36963991014929), Diff(x) = 22.86492417938325
Step 8: x = (50.4513236997866, 50.423176944750026), Diff(x) = 22.854549238111588
Step 9: x = (50.50954542688473, 50.476905832759655), Diff(x) = 22.844059302911766
Step 10: x = (50.568187192374126, 50.530829771733465), Diff(x) = 22.83345194303347
Step 11: x = (50.62725499747032, 50.58495204197229), Diff(x) = 22.822724650547514
Step 12: x = (50.68675500490869, 50.63927600972207), Diff(x) = 22.811874836991734
Step 13: x = (50.7466

# 8. Repeat step 7 with starting points (0,0) and (100,100). 
**Do you always end up at the same point?**

In [75]:
#interpretation still missing. " Do you always end up at the same point? " 
stepsize1 = 0.8# 1.65 seems good but you're free to check further. 
stepsize2 = 0.5 # stepsize still needs to be determined
delta = 0.001
iters = 600
point1 = (0,0)
point2 = (100,100)
g_descent(point1, stepsize1,delta, iters)
g_descent(point2, stepsize2,delta, iters)

Step 1: x = (0.04727983565260274, 0.053538505409278514), Diff(x) = 27.706105550498528
Step 2: x = (0.09455774217315138, 0.1071647323385605), Diff(x) = 27.699713929944487
Step 3: x = (0.14183354599026643, 0.16087897509180493), Diff(x) = 27.69331073868863
Step 4: x = (0.18910707190116227, 0.2146815293514237), Diff(x) = 27.68689593823879
Step 5: x = (0.23637814306312066, 0.26857269221522984), Diff(x) = 27.680469489907562
Step 6: x = (0.28364658097927986, 0.32255276219927964), Diff(x) = 27.674031354811063
Step 7: x = (0.33091220547873945, 0.37662203923503057), Diff(x) = 27.66758149386844
Step 8: x = (0.37817483470234947, 0.4307808246892364), Diff(x) = 27.6611198677994
Step 9: x = (0.4254342850828152, 0.4850294213667894), Diff(x) = 27.654646437123287
Step 10: x = (0.4726903713390129, 0.5393681335192468), Diff(x) = 27.648161162156843
Step 11: x = (0.519942906447568, 0.5937972668647262), Diff(x) = 27.64166400301257
Step 12: x = (0.5671917016343286, 0.6483171285822209), Diff(x) = 27.6351549195

# 9. Make the nicest possible chart using mathplotlib that shows what the Diff function looks like.

# 10. Show the gradient field as well in a chart. 
**Use a new plot or add the gradient to the original plot.**